# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,373.910394,0.305586,1234,374.215980,373.604808,748.431960,747.209616
6,308.350207,-0.568020,1234,307.782187,308.918228,615.564374,617.836455
7,653.852199,-0.684541,1234,653.167658,654.536740,1306.335316,1309.073480
8,926.952109,-0.417584,1234,926.534525,927.369692,1853.069050,1854.739385
9,606.207713,0.028116,1234,606.235830,606.179597,1212.471659,1212.359194
10,200.883160,-0.670089,1234,200.213071,201.553249,400.426142,403.106499
11,140.600031,-0.382574,1234,140.217457,140.982605,280.434915,281.965209
12,822.243884,-0.884637,1234,821.359247,823.128522,1642.718494,1646.257043
13,757.271042,0.565761,1234,757.836803,756.705281,1515.673606,1513.410562
14,764.992084,-0.721201,1234,764.270883,765.713285,1528.541767,1531.426570


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-374.215980,373.604808
6,-307.782187,308.918228
7,-653.167658,654.536740
8,-926.534525,927.369692
9,-606.235830,606.179597
10,-200.213071,201.553249
11,-140.217457,140.982605
12,-821.359247,823.128522
13,-757.836803,756.705281
14,-764.270883,765.713285


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-374.215980,373.604808
6,-307.782187,308.918228
7,-653.167658,654.536740
8,-926.534525,927.369692
9,-606.235830,606.179597
10,-200.213071,201.553249
11,-140.217457,140.982605
12,-821.359247,823.128522
13,-757.836803,756.705281
14,-764.270883,765.713285


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-374.21598024,  373.60480812],
       [-307.78218685,  308.91822765],
       [-653.16765788,  654.53674007],
       [-926.53452501,  927.36969243],
       [-606.23582953,  606.17959687],
       [-200.2130709 ,  201.55324937],
       [-140.21745728,  140.98260458],
       [-821.35924703,  823.1285216 ],
       [-757.83680309,  756.70528117],
       [-764.27088344,  765.71328487]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,746.431960,-747.209616
6,613.564374,-617.836455
7,1304.335316,-1309.073480
8,1851.069050,-1854.739385
9,1210.471659,-1212.359194
10,398.426142,-403.106499
11,278.434915,-281.965209
12,1640.718494,-1646.257043
13,1513.673606,-1513.410562
14,1526.541767,-1531.426570


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [16]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [17]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [18]:
data['a']  = np.nan

In [19]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [20]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [21]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
